In [1]:
import scipy.stats as st
from math import floor, ceil
#df=dataframe, index_list=list of names of the 13 genes, lam = dictionary of parameter for the poisson with keys corresponding to index_list 
def expected_count_poi(df, index_list, lam):
    expected_vals=[] #frequency
    expected_ranges=[] #corresponding ranges
    n=len(df)
    for j in index_list:
        li=[]
        ind=[]
        lamb=lam[j]
        m=floor(min(df[j]))
        M=ceil(max(df[j]))
        low=m
        upp=m+1
        if m==0:
            coun=n*st.poisson.cdf(m+1,lamb) 
        else: 
            coun=n*(st.poisson.cdf(m+1,lamb)-st.poisson.cdf(m-1,lamb))
        if coun>=5:
            ind.append([low,upp])
            li.append(coun)
            low=upp
            upp+=1
            coun=0
        else:
            pass
        for i in range(m+1,M-1,1):
            coun+=n*(st.poisson.cdf(i+1,lamb)-st.poisson.cdf(i,lamb))
            if coun>=5: 
                upp=i+1
                ind.append([low,upp])
                li.append(coun)
                coun=0
                low=upp
            else:
                pass
        upp=M
        coun+=n*(st.poisson.cdf(M,lamb)-st.poisson.cdf(M-1,lamb))
        if coun>=5:
            ind.append([low,upp])
            li.append(coun)
        else:
            li[-1]=li[-1]+coun
            ind[-1][1]=upp
        expected_vals.append(li)
        expected_ranges.append(ind)
    expected_vals=dict(zip(index_list,expected_vals))
    expected_ranges=dict(zip(index_list,expected_ranges))
    return (expected_vals,expected_ranges)

def ex_vals_poi(df, index_list, lam):
    return expected_count_poi(df, index_list, lam)[0]

def ex_range_poi(df, index_list, lam):
    return expected_count_poi(df, index_list, lam)[1]

In [2]:
def obs_count_poi(df,index_list, lam):
    comp_ran=ex_range_poi(df,index_list, lam)
    obs_val=[]
    for j in index_list:
        li=[]
        ran=comp_ran[j]
        df_j=df[j]
        l,h=ran[0][0],ran[0][1]
        li.append(len(df_j[(df_j>=l) & (df_j<=h)]))
        for i in range(1,len(ran),1):
            l,h=ran[i][0],ran[i][1]
            li.append(len(df_j[(df_j>l) & (df_j<=h)]))
        obs_val.append(li)
    return dict(zip(index_list,obs_val))
    

In [4]:
def chi2_res_poi(df,index_list, lam, alp=0.95):
    ex=ex_vals_poi(df, index_list, lam)
    obs=obs_count_poi(df,index_list, lam)
    import scipy.stats as st
    res=[]
    for j in index_list:
        deg=len(ex)-1
        cv=st.chi2.ppf(alp,deg)
        ele=[(a-b)**2/a for a,b in zip(ex[j],obs[j])]
        ts=sum(ele)
        res.append(ts-cv)
    return res



In [ ]:
import scipy.stats as st
from math import floor, ceil
#df=dataframe, index_list=list of names of the 13 genes, prob = dictionary of prob parameter nbin for the  with keys corresponding to index_list, r= uniform no. of success
def expected_count_nbi(df, index_list, prob,r=100):
    expected_vals=[] #frequency
    expected_ranges=[] #corresponding ranges
    n=len(df)
    for j in index_list:
        li=[]
        ind=[]
        p=prob[j]
        m=floor(min(df[j]))
        M=ceil(max(df[j]))
        low=m
        upp=m+1
        if m==0:
            coun=n*st.nbinom.cdf(m+1,r,p) 
        else: 
            coun=n*(st.nbinom.cdf(m+1,r,p)-st.nbinom.cdf(m-1,r,p))
        if coun>=5:
            ind.append([low,upp])
            li.append(coun)
            low=upp
            upp+=1
            coun=0
        else:
            pass
        for i in range(m+1,M-1,1):
            coun+=n*(st.nbinom.cdf(i+1,r,p)-st.nbinom.cdf(i,r,p))
            if coun>=5: 
                upp=i+1
                ind.append([low,upp])
                li.append(coun)
                coun=0
                low=upp
            else:
                pass
        upp=M
        coun+=n*(st.nbinom.cdf(M,r,p)-st.nbinom.cdf(M-1,r,p))
        if coun>=5:
            ind.append([low,upp])
            li.append(coun)
        else:
            li[-1]=li[-1]+coun
            ind[-1][1]=upp
        expected_vals.append(li)
        expected_ranges.append(ind)
    expected_vals=dict(zip(index_list,expected_vals))
    expected_ranges=dict(zip(index_list,expected_ranges))
    return (expected_vals,expected_ranges)

def ex_vals_nbi(df, index_list, prob,r=100):
    return expected_count_nbi(df, index_list, prob,r)[0]

def ex_range_nbi(df, index_list, prob,r=100):
    return expected_count_nbi(df, index_list, prob,r)[1]

In [ ]:
def obs_count_nbi(df,index_list, prob, r=100):
    comp_ran=ex_range_nbi(df,index_list, prob, r)
    obs_val=[]
    for j in index_list:
        li=[]
        ran=comp_ran[j]
        df_j=df[j]
        l,h=ran[0][0],ran[0][1]
        li.append(len(df_j[(df_j>=l) & (df_j<=h)]))
        for i in range(1,len(ran),1):
            l,h=ran[i][0],ran[i][1]
            li.append(len(df_j[(df_j>l) & (df_j<=h)]))
        obs_val.append(li)
    return dict(zip(index_list,obs_val))
    

In [ ]:
def chi2_res_nbi(df,index_list, prob, r=100, alp=0.99):
    ex=ex_vals_nbi(df, index_list, prob)
    obs=obs_count_nbi(df,index_list, prob)
    import scipy.stats as st
    res=[]
    for j in index_list:
        deg=len(ex)-1
        cv=st.chi2.ppf(alp,deg)
        ele=[(a-b)**2/a for a,b in zip(ex[j],obs[j])]
        ts=sum(ele)
        res.append(ts-cv)
    return res



In [7]:
#df: dataframe, index_list= list of columns, r=success paramter of nbinom, prob=list of probabilities with keys from index_lsit
def qq_nbi(df, index_list , r , prob):
    for i in index_list:
    # Sort the data
        data = np.sort(df[i])
        p=prob[i]
    # Generate the theoretical quantiles from the negative binomial distribution
        quant = np.arange(1, len(data) + 1) / (len(data) + 1)
        nbi_quant = st.nbinom.ppf(quant, r, p)
    
    # Create the Q-Q plot
        fig,ax=plt.subplots(1,1)
        plt.figure(figsize=(8, 8))
        ax.scatter(nbi_quant, data, edgecolor='k', facecolor='none')
        ax.plot([min(nbi_quant), max(nbi_quant)], 
             [min(nbi_quant), max(nbi_quant)], 
             color='red', linestyle='--')
        ax.set_xlabel('Theoretical Quantiles (Negative Binomial)')
        ax.set_ylabel( i + ' Sample' 'Quantiles')
        ax.set_title('Q-Q 1_4:' + i+  ' vs Negative Binomial Distribution')
        ax.grid(True)
        plt.show()
